# Лабораторная работа 4

Tensorflow 2.x

1) Подготовка данных

2) Использование Keras Model API

3) Использование Keras Sequential + Functional API

https://www.tensorflow.org/tutorials

Для выполнения лабораторной работы необходимо установить tensorflow версии 2.0 или выше .

Рекомендуется использовать возможности Colab'а по обучению моделей на GPU.



In [2]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

%matplotlib inline

In [14]:
USE_GPU = False
device = '/device:GPU:0' if USE_GPU else '/cpu:0'

In [4]:
print_every = 100

# Подготовка данных
Загрузите набор данных из предыдущей лабораторной работы.

In [21]:
def load_mnist(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    mnist = tf.keras.datasets.mnist.load_data()
    (X_train, y_train), (X_test, y_test) = mnist
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test

# If there are errors with SSL downloading involving self-signed certificates,
# it may be that your Python version was recently installed on the current machine.
# See: https://github.com/tensorflow/tensorflow/issues/10779
# To fix, run the command: /Applications/Python\ 3.7/Install\ Certificates.command
#   ...replacing paths as necessary.

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_mnist()
X_train = X_train.reshape(-1, 28, 28, 1)
X_val = X_val.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (49000, 28, 28, 1)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 28, 28, 1)
Validation labels shape:  (1000,)
Test data shape:  (10000, 28, 28, 1)
Test labels shape:  (10000,)


In [24]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y

        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [25]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 28, 28, 1) (64,)
1 (64, 28, 28, 1) (64,)
2 (64, 28, 28, 1) (64,)
3 (64, 28, 28, 1) (64,)
4 (64, 28, 28, 1) (64,)
5 (64, 28, 28, 1) (64,)
6 (64, 28, 28, 1) (64,)


#  Keras Model Subclassing API


Для реализации собственной модели с помощью Keras Model Subclassing API необходимо выполнить следующие шаги:

1) Определить новый класс, который является наследником tf.keras.Model.

2) В методе __init__() определить все необходимые слои из модуля tf.keras.layer

3) Реализовать прямой проход в методе call() на основе слоев, объявленных в __init__()

Ниже приведен пример использования keras API для определения двухслойной полносвязной сети.

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras

In [26]:
class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super(TwoLayerFC, self).__init__()
        initializer = tf.initializers.VarianceScaling(scale=2.0, seed=42)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu',
                                   kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()

    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    x = tf.zeros((64, input_size))
    model = TwoLayerFC(hidden_size, num_classes)
    with tf.device(device):
        scores = model(x)
        print(scores.shape)

test_TwoLayerFC()

(64, 10)


Реализуйте трехслойную CNN для вашей задачи классификации.

Архитектура сети:
    
1. Сверточный слой (5 x 5 kernels, zero-padding = 'same')
2. Функция активации ReLU
3. Сверточный слой (3 x 3 kernels, zero-padding = 'same')
4. Функция активации ReLU
5. Полносвязный слой
6. Функция активации Softmax

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Conv2D

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense

In [27]:
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()
        ########################################################################
        # TODO: Implement the __init__ method for a three-layer ConvNet. You   #
        # should instantiate layer objects to be used in the forward pass.     #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        self.conv1 = tf.keras.layers.Conv2D(channel_1, (5, 5), padding='same', activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(channel_2, (3, 3), padding='same', activation='relu')
        self.flatten = tf.keras.layers.Flatten()
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################

    def call(self, x, training=False):
        scores = None
        ########################################################################
        # TODO: Implement the forward pass for a three-layer ConvNet. You      #
        # should use the layer objects defined in the __init__ method.         #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        scores = self.conv1(x)
        scores = self.conv2(scores)
        scores = self.flatten(scores)
        scores = self.fc(scores)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################
        return scores

In [28]:
def test_ThreeLayerConvNet():
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    with tf.device(device):
        x = tf.zeros((64, 3, 32, 32))
        scores = model(x)
        print(scores.shape)

test_ThreeLayerConvNet()

(64, 10)


Пример реализации процесса обучения:

In [30]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.

    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for

    Returns: Nothing, but prints progress during trainingn
    """
    with tf.device(device):
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

        model = model_init_fn()
        optimizer = optimizer_init_fn()

        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')

        t = 0
        for epoch in range(num_epochs):

            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            train_loss.reset_states()
            train_accuracy.reset_states()

            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:

                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)

                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)

                    if t % print_every == 0:
                        val_loss.reset_states()
                        val_accuracy.reset_states()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)

                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

In [31]:
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return TwoLayerFC(hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.061091423034668, Accuracy: 7.8125, Val Loss: 2.6353776454925537, Val Accuracy: 20.100000381469727
Iteration 100, Epoch 1, Loss: 0.6486901640892029, Accuracy: 80.30631256103516, Val Loss: 0.5465362668037415, Val Accuracy: 83.20000457763672
Iteration 200, Epoch 1, Loss: 0.5134939551353455, Accuracy: 84.65484619140625, Val Loss: 0.44879984855651855, Val Accuracy: 86.29999542236328
Iteration 300, Epoch 1, Loss: 0.45332732796669006, Accuracy: 86.5188980102539, Val Loss: 0.4038299024105072, Val Accuracy: 87.30000305175781
Iteration 400, Epoch 1, Loss: 0.40937238931655884, Accuracy: 87.87017059326172, Val Loss: 0.3598787486553192, Val Accuracy: 89.20000457763672
Iteration 500, Epoch 1, Loss: 0.3856959044933319, Accuracy: 88.60404205322266, Val Loss: 0.34413355588912964, Val Accuracy: 90.20000457763672
Iteration 600, Epoch 1, Loss: 0.361906498670578, Accuracy: 89.31208038330078, Val Loss: 0.33807024359703064, Val Accuracy: 90.80000305175781
Iteration 700, Epoch 1,

Обучите трехслойную CNN. В tf.keras.optimizers.SGD укажите Nesterov momentum = 0.9 .

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/optimizers/SGD

Значение accuracy на валидационной выборке после 1 эпохи обучения должно быть > 50% .

In [32]:
learning_rate = 3e-3
channel_1, channel_2, num_classes = 32, 16, 10

def model_init_fn():
    model = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return model

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.3081676959991455, Accuracy: 4.6875, Val Loss: 2.2887723445892334, Val Accuracy: 10.100000381469727
Iteration 100, Epoch 1, Loss: 0.6771867275238037, Accuracy: 80.07425689697266, Val Loss: 0.4905747175216675, Val Accuracy: 83.70000457763672
Iteration 200, Epoch 1, Loss: 0.5065358281135559, Accuracy: 85.3078384399414, Val Loss: 0.3975633382797241, Val Accuracy: 87.69999694824219
Iteration 300, Epoch 1, Loss: 0.4143221974372864, Accuracy: 87.92566680908203, Val Loss: 0.23670873045921326, Val Accuracy: 93.30000305175781
Iteration 400, Epoch 1, Loss: 0.349885493516922, Accuracy: 89.73270416259766, Val Loss: 0.22762106359004974, Val Accuracy: 92.69999694824219
Iteration 500, Epoch 1, Loss: 0.31137359142303467, Accuracy: 90.89633178710938, Val Loss: 0.1932782381772995, Val Accuracy: 93.9000015258789
Iteration 600, Epoch 1, Loss: 0.27978163957595825, Accuracy: 91.77932739257812, Val Loss: 0.20306076109409332, Val Accuracy: 93.4000015258789
Iteration 700, Epoch 1, 

# Использование Keras Sequential API для реализации последовательных моделей.

Пример для полносвязной сети:

In [34]:
learning_rate = 1e-2

def model_init_fn():
    input_shape = (28, 28, 1)
    hidden_layer_size, num_classes = 4000, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu',
                              kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax',
                              kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.0636892318725586, Accuracy: 9.375, Val Loss: 2.8174750804901123, Val Accuracy: 20.600000381469727
Iteration 100, Epoch 1, Loss: 0.6536769270896912, Accuracy: 80.81682586669922, Val Loss: 0.5527722835540771, Val Accuracy: 82.0
Iteration 200, Epoch 1, Loss: 0.5154412984848022, Accuracy: 85.02798461914062, Val Loss: 0.4694288671016693, Val Accuracy: 85.9000015258789
Iteration 300, Epoch 1, Loss: 0.4560350179672241, Accuracy: 86.73172760009766, Val Loss: 0.41941115260124207, Val Accuracy: 87.0
Iteration 400, Epoch 1, Loss: 0.41096195578575134, Accuracy: 88.06889343261719, Val Loss: 0.38207271695137024, Val Accuracy: 89.0999984741211
Iteration 500, Epoch 1, Loss: 0.38665854930877686, Accuracy: 88.7724609375, Val Loss: 0.3638768196105957, Val Accuracy: 89.60000610351562
Iteration 600, Epoch 1, Loss: 0.3633613884449005, Accuracy: 89.39008331298828, Val Loss: 0.34729209542274475, Val Accuracy: 90.30000305175781
Iteration 700, Epoch 1, Loss: 0.34644562005996704, Ac

Альтернативный менее гибкий способ обучения:

In [35]:
model = model_init_fn()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)



313/313 [==============================] - 2s 6ms/step - loss: 0.2242 - sparse_categorical_accuracy: 0.9346


[0.2242462933063507, 0.9345999956130981]

Перепишите реализацию трехслойной CNN с помощью tf.keras.Sequential API . Обучите модель двумя способами.

In [36]:
def model_init_fn():
    model = None
    ############################################################################
    # TODO: Construct a three-layer ConvNet using tf.keras.Sequential.         #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = tf.keras.Sequential([
        # The first conv layer
        tf.keras.layers.Conv2D(32, (5, 5), padding='same', activation='relu', input_shape=(28, 28, 1)),
        # The first pooling layer
        tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),
        # The second conv layer
        tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='relu'),
        # The second pooling layer
        tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),
        # Flatten data
        tf.keras.layers.Flatten(),
        # The dense layer with ReLU
        tf.keras.layers.Dense(1024, activation='relu'),
        # Output layer with softmax 
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    return model

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                            END OF YOUR CODE                              #
    ############################################################################
    return model

learning_rate = 5e-4
def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)


Iteration 0, Epoch 1, Loss: 2.3050031661987305, Accuracy: 12.5, Val Loss: 2.3038103580474854, Val Accuracy: 10.59999942779541
Iteration 100, Epoch 1, Loss: 2.2718451023101807, Accuracy: 12.979578971862793, Val Loss: 2.2375638484954834, Val Accuracy: 17.399999618530273
Iteration 200, Epoch 1, Loss: 2.2371981143951416, Accuracy: 18.11256217956543, Val Loss: 2.1753196716308594, Val Accuracy: 28.400001525878906
Iteration 300, Epoch 1, Loss: 2.2043821811676025, Accuracy: 25.47757339477539, Val Loss: 2.110020160675049, Val Accuracy: 46.599998474121094
Iteration 400, Epoch 1, Loss: 2.1681740283966064, Accuracy: 32.8125, Val Loss: 2.032646894454956, Val Accuracy: 57.099998474121094
Iteration 500, Epoch 1, Loss: 2.1296353340148926, Accuracy: 39.115516662597656, Val Loss: 1.942175030708313, Val Accuracy: 61.19999694824219
Iteration 600, Epoch 1, Loss: 2.0857419967651367, Accuracy: 44.10877990722656, Val Loss: 1.8324133157730103, Val Accuracy: 65.69999694824219
Iteration 700, Epoch 1, Loss: 2.03

In [37]:
model = model_init_fn()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)


313/313 [==============================] - 5s 16ms/step - loss: 0.1210 - sparse_categorical_accuracy: 0.9669


[0.12101379036903381, 0.9668999910354614]

# Использование Keras Functional API

Для реализации более сложных архитектур сети с несколькими входами/выходами, повторным использованием слоев, "остаточными" связями (residual connections) необходимо явно указать входные и выходные тензоры.

Ниже представлен пример для полносвязной сети.

In [38]:
def two_layer_fc_functional(input_shape, hidden_size, num_classes):
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    inputs = tf.keras.Input(shape=input_shape)
    flattened_inputs = tf.keras.layers.Flatten()(inputs)
    fc1_output = tf.keras.layers.Dense(hidden_size, activation='relu',
                                 kernel_initializer=initializer)(flattened_inputs)
    scores = tf.keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer=initializer)(fc1_output)

    # Instantiate the model given inputs and outputs.
    model = tf.keras.Model(inputs=inputs, outputs=scores)
    return model

def test_two_layer_fc_functional():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    input_shape = (50,)

    x = tf.zeros((64, input_size))
    model = two_layer_fc_functional(input_shape, hidden_size, num_classes)

    with tf.device(device):
        scores = model(x)
        print(scores.shape)

test_two_layer_fc_functional()

(64, 10)


In [39]:
input_shape = (28, 28, 1)
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return two_layer_fc_functional(input_shape, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.691138744354248, Accuracy: 9.375, Val Loss: 2.4658114910125732, Val Accuracy: 18.299999237060547
Iteration 100, Epoch 1, Loss: 0.6254281997680664, Accuracy: 81.0643539428711, Val Loss: 0.5445429682731628, Val Accuracy: 81.9000015258789
Iteration 200, Epoch 1, Loss: 0.5011837482452393, Accuracy: 84.98912048339844, Val Loss: 0.4637463688850403, Val Accuracy: 85.69999694824219
Iteration 300, Epoch 1, Loss: 0.44794031977653503, Accuracy: 86.64867401123047, Val Loss: 0.40109431743621826, Val Accuracy: 88.30000305175781
Iteration 400, Epoch 1, Loss: 0.40599456429481506, Accuracy: 87.93640899658203, Val Loss: 0.35908541083335876, Val Accuracy: 89.60000610351562
Iteration 500, Epoch 1, Loss: 0.38384509086608887, Accuracy: 88.61651611328125, Val Loss: 0.34215793013572693, Val Accuracy: 90.30000305175781
Iteration 600, Epoch 1, Loss: 0.36028924584388733, Accuracy: 89.35368347167969, Val Loss: 0.33608728647232056, Val Accuracy: 90.80000305175781
Iteration 700, Epoch 

Поэкспериментируйте с архитектурой сверточной сети. Для вашего набора данных вам необходимо получить как минимум 70% accuracy на валидационной выборке за 10 эпох обучения. Опишите все эксперименты и сделайте выводы (без выполнения данного пункта работы приниматься не будут).

Эспериментируйте с архитектурой, гиперпараметрами, функцией потерь, регуляризацией, методом оптимизации.  

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/BatchNormalization#methods https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dropout#methods

In [44]:
class CustomConvNet(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        self.conv1 = tf.keras.layers.Conv2D(28, (3, 3), activation='relu', input_shape=(28, 28, 1))

        self.batchnorm1 = tf.keras.layers.BatchNormalization()

        self.conv2 = tf.keras.layers.Conv2D(14, (3, 3), activation='relu')

        self.batchnorm2 = tf.keras.layers.BatchNormalization()

        self.dense1 = tf.keras.layers.Dense(56, activation='relu')
        
        self.dropout1 = tf.keras.layers.Dropout(0.2)
        
        self.flatten = tf.keras.layers.Flatten()

        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################

    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(input_tensor)
        x = self.batchnorm1(x)

        x = self.conv2(x)
        x = self.batchnorm2(x)
        x = self.dense1(x)

        x = self.dropout1(x, training=training)

        x = self.flatten(x)
        x = self.dense2(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        return x


print_every = 700
num_epochs = 10

model = CustomConvNet()

def model_init_fn():
    return CustomConvNet()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate)

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 2.351034641265869, Accuracy: 15.625, Val Loss: 2.2210841178894043, Val Accuracy: 19.100000381469727
Iteration 700, Epoch 1, Loss: 0.1622346192598343, Accuracy: 95.27461242675781, Val Loss: 0.1292656660079956, Val Accuracy: 96.0
Iteration 1400, Epoch 2, Loss: 0.04549592360854149, Accuracy: 98.59497833251953, Val Loss: 0.09731973707675934, Val Accuracy: 96.5999984741211
Iteration 2100, Epoch 3, Loss: 0.03049553744494915, Accuracy: 99.00592803955078, Val Loss: 0.1156710684299469, Val Accuracy: 97.19999694824219
Iteration 2800, Epoch 4, Loss: 0.02283533103764057, Accuracy: 99.1861343383789, Val Loss: 0.09055744856595993, Val Accuracy: 97.69999694824219
Iteration 3500, Epoch 5, Loss: 0.012596665881574154, Accuracy: 99.56736755371094, Val Loss: 0.0892658531665802, Val Accuracy: 97.69999694824219
Iteration 4200, Epoch 6, Loss: 0.01604273170232773, Accuracy: 99.4609146118164, Val Loss: 0.0874805897474289, Val Accuracy: 97.39999389648438
Iteration 4900, Epoch 7, Loss

Опишите все эксперименты, результаты. Сделайте выводы.

1. Архитектура сети состоит из: сверточных слоев, droput, flatten, batchnormalization и полносвязных слоев.
2. Для выходного слоя тестировались функции активации softmax и sigmoid. Первая показала наилучший результат.
3. Среди оптимизаторов лучше всего себя показал Adam - незначительно, но лучше.
4. Также модель тестировалась на смене batchnormalization и droput. Модели имели следующие слои:
    1. dropuot и batchnormalization (показал наилучшие результаты).
    2. droput.
    3. batchnormalization.
    4. Ни одного .
5. Худшим показателем получения accuracy > 70 было 5 эпох.
6. В качестве замены ReLU рассматривалась sigmoid, но первая оказала наилучшее влияние.
7. По итогам 10 эпох точность модели на валидационной выборке составила более 97%.